# Lab 3: AgentCore Code Interpreter を使用した修復エージェント

## 概要
セキュアなインフラストラクチャ自動化とスクリプト実行のために AgentCore Code Interpreter を使用する Strands ベースの修復エージェントを構築します。

## 目標
- AgentCore Code Interpreter 統合を持つ Strands エージェントを作成
- 修復プラン生成とセキュアなスクリプト実行のためのツールを構築
- インフラストラクチャ修復シナリオでエージェントをテスト
- アクセスと実行の安全性を検証

## 学習内容
- AgentCore Code Interpreter を Strands エージェントと統合する方法
- 適切な制御を持つセキュアな修復ワークフローの実装方法
- インフラストラクチャ自動化用のツールを作成する方法
- エージェントの修復計画と実行パターン

## アーキテクチャ概要
```
┌─────────────────┐    ┌──────────────────────┐    ┌─────────────────────┐
│   ユーザーリクエスト │───▶│  Strands Agent       │───▶│  AgentCore Code     │
│                 │    │  （修復）             │    │  Interpreter        │
└─────────────────┘    └──────────────────────┘    └─────────────────────┘
                                │                            │
                                ▼                            ▼
                       ┌──────────────────────┐    ┌─────────────────────┐
                       │  修復ツール           │    │  セキュアな Python   │
                       │  ├─ プラン生成        │    │  実行               │
                       │  ├─ レビューゲート    │    │  ├─ セッション管理   │
                       │  └─ スクリプト実行    │    │  ├─ コードストリーミング │
                       └──────────────────────┘    │  └─ エラー処理       │
                                │                  └─────────────────────┘
                                ▼                            │
                       ┌──────────────────────┐              ▼
                       │  インフラストラクチャ  │    ┌─────────────────────┐
                       │  変更                │◀───│  実行結果           │
                       │  （承認済みのみ）      │    │  と検証             │
                       └──────────────────────┘    └─────────────────────┘
```

**主要コンポーネント:**
- **2段階プロセス**: 計画 → レビュー → 実行
- **セキュアな実行**: AgentCore Code Interpreter が分離された環境を提供
- **リスク評価**: 各ステップの包括的な影響分析

## 0. 必要なパッケージのインストール

このセルを最初に実行して、すべての依存関係がインストールされていることを確認してください。

In [ ]:
#%pip install -q -r requirements.txt
print("✅ ワークショップの依存関係がインストールされました")

## 1. 必要なモジュールのインポート

In [ ]:
### 1. インポート

# AWS SDK と設定
import json
import boto3
import logging
import uuid
from datetime import datetime
from typing import Dict, List, Optional

# Strands フレームワーク
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool

# Bedrock AgentCore スターターツールキット
from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.config import AWS_REGION, WORKSHOP_NAME
from lab_helpers.parameter_store import get_parameter, put_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.lab_03.gateway_setup import AgentCoreGatewaySetup

# ワークショップ設定
from botocore.config import Config
from lab_helpers.config import MODEL_ID, AWS_REGION, AWS_PROFILE, WORKSHOP_NAME

# Lab-03 デプロイヘルパー
from lab_helpers.lab_03 import (
    AgentCoreRuntimeDeployer,
    AgentCoreGatewaySetup,
    cleanup_lab_03
)
# ファイルシステム操作
from pathlib import Path

# ノートブック用のロギングを設定
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

# Code Interpreter 用のグローバル変数
agentcore_code_interpreter = None
CODE_INTERPRETER_AVAILABLE = False

print("✅ インポートが読み込まれました")
print(f"   ワークショップ: {WORKSHOP_NAME}")
print(f"   リージョン: {AWS_REGION}")
print(f"   モデル: {MODEL_ID}")

## 2. 前提条件のセットアップと確認

In [ ]:
# 修復プラン用の S3 バケットを作成
import boto3
import uuid
from botocore.exceptions import ClientError
from lab_helpers.config import AWS_REGION

s3_client = boto3.client('s3', region_name=AWS_REGION)
unique_suffix = str(uuid.uuid4())[:8]
bucket_name = f'sre-workshop-remediation-plans-{unique_suffix}'

try:
    # us-east-1 は LocationConstraint を使用しない
    if AWS_REGION == 'us-east-1':
        response = s3_client.create_bucket(Bucket=bucket_name)
    else:
        response = s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': AWS_REGION}
        )
    
    print(f"✅ バケットを作成しました: {bucket_name}")
    
    # Parameter Store に保存
    ssm = boto3.client('ssm', region_name=AWS_REGION)
    ssm.put_parameter(
        Name='/aiml301_sre_workshop/remediation_s3_bucket',
        Value=bucket_name,
        Type='String',
        Overwrite=True
    )
    print(f"✅ Parameter Store に保存しました")
    
except ClientError as e:
    print(f"❌ エラー: {e}")



try:
    # AWS 認証情報をテスト
    sts_client = boto3.client('sts', region_name=AWS_REGION)
    identity = sts_client.get_caller_identity()
    account_id = identity['Account']
    
    # AgentCore Code Interpreter の利用可能性をテスト
    agentcore_test = boto3.client('bedrock-agentcore', region_name=AWS_REGION)
    
    print(f"✅ 前提条件を確認しました: AWS アカウント {account_id}、AgentCore Code Interpreter が利用可能です")
    print(f"   リージョン: {AWS_REGION}")
    print(f"   プロファイル: {AWS_PROFILE}")
    print(f"   モデル ID: {MODEL_ID}")
    print(f"   ID: {identity.get('Arn', 'Unknown')}")
    
except Exception as e:
    print(f"❌ エラー: {e}")
    print("AWS 認証情報が設定されており、AgentCore Code Interpreter の権限が利用可能であることを確認してください。")

In [ ]:
# SSM Parameter Store に保存
parameter_name = '/aiml301_sre_workshop/remediation_s3_bucket'
ssm = boto3.client('ssm', region_name='us-west-2')
parameter = ssm.get_parameter(Name=parameter_name)
retrieved_bucket_name = parameter['Parameter']['Value']

## 3. カスタム Code Interpreter のセットアップ

**目標:** カスタム IAM 実行ロールを持つカスタム AgentCore Code Interpreter を作成します。

**アプローチ:**
1. 適切な信頼ポリシーと権限を持つカスタム IAM 実行ロールを作成
2. PUBLIC ネットワークモードでカスタム Code Interpreter を作成
3. クライアントとセッション管理関数を初期化

**重要な学習ポイント:** ユースケースに応じた特定の権限を持つカスタム Code Interpreter の作成方法。

### 3.1 カスタム Code Interpreter 用の IAM ポリシーとロールのセットアップ

In [ ]:
### 3.1: カスタム IAM 実行ロールを作成

def create_custom_code_interpreter_role():
    """カスタム Code Interpreter 用の IAM 実行ロールを作成"""
    iam_client = boto3.client('iam')
    sts_client = boto3.client('sts')
    account_id = sts_client.get_caller_identity()['Account']
    
    role_name = f"{WORKSHOP_NAME}-CodeInterpreterRole"
    
    # 信頼ポリシー - bedrock-agentcore サービスがロールを引き受けることを許可
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Sid": "AssumeRolePolicy",
            "Effect": "Allow",
            "Principal": {"Service": "bedrock-agentcore.amazonaws.com"},
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {"aws:SourceAccount": account_id},
                "ArnLike": {"aws:SourceArn": f"arn:aws:bedrock-agentcore:{AWS_REGION}:{account_id}:*"}
            }
        }]
    }
    
    # Code Interpreter 実行用の権限ポリシー
    with open('lab_helpers/lab_03/code_interpreter_permissions_policy.json', 'r') as f:
        ci_permissions_policy = f.read()
        ci_permissions_policy = ci_permissions_policy.replace('{{ACCOUNT_ID}}', account_id)
        ci_permissions_policy = ci_permissions_policy.replace('{{REGION}}', AWS_REGION)
    
    try:
        # ロールを作成
        response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description="Custom execution role for AgentCore Code Interpreter",
            Tags=[{'Key': 'Workshop', 'Value': WORKSHOP_NAME}]
        )
        role_arn = response['Role']['Arn']
        logger.info(f"✅ IAM ロールを作成しました: {role_name}")
    except iam_client.exceptions.EntityAlreadyExistsException:
        response = iam_client.get_role(RoleName=role_name)
        role_arn = response['Role']['Arn']
        logger.info(f"✅ 既存の IAM ロールを使用: {role_name}")
    
    # 権限ポリシーをアタッチ
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName='CodeInterpreterExecutionPolicy',
            PolicyDocument=ci_permissions_policy
        )
        logger.info("✅ 権限ポリシーをアタッチしました")
    except Exception as e:
        logger.warning(f"ポリシーが既に存在する可能性があります: {e}")
    
    return role_arn, role_name

# ロールを作成
print("🔧 カスタム IAM 実行ロールを作成中...")
custom_role_arn, custom_role_name = create_custom_code_interpreter_role()
print(f"✅ IAM ロール: {custom_role_name}")
print(f"   ARN: {custom_role_arn}")

### 3.2 AgentCore カスタム Code Interpreter の作成

In [ ]:
### 3.2: カスタム Code Interpreter を作成

import time

def create_custom_code_interpreter(role_arn):
    """実行ロール付きのカスタム Code Interpreter を作成"""
    agentcore_control = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)
    
    interpreter_name = f"{WORKSHOP_NAME}_custom_code_interpreter"
    
    try:
        response = agentcore_control.create_code_interpreter(
            name=interpreter_name,
            executionRoleArn=role_arn,
            networkConfiguration={"networkMode": "PUBLIC"},
            description="Custom code interpreter for remediation agent",
            clientToken=str(uuid.uuid4())
        )
        interpreter_id = response['codeInterpreterId']
        interpreter_arn = response['codeInterpreterArn']
        status = response['status']
        logger.info(f"✅ Code Interpreter を作成しました: {interpreter_id}")
        
    except agentcore_control.exceptions.ConflictException:
        logger.info(f"⚠️  Code Interpreter が既に存在します。検索中...")
        # リストで実際の ID を見つける（キーは codeInterpreters ではなく codeInterpreterSummaries）
        list_response = agentcore_control.list_code_interpreters()
        all_items = list_response.get('codeInterpreterSummaries', [])
        
        interpreter_id = None
        for item in all_items:
            if item.get('name') == interpreter_name:
                interpreter_id = item['codeInterpreterId']
                interpreter_arn = item['codeInterpreterArn']
                status = item['status']
                logger.info(f"✅ 既存の Interpreter を発見: {interpreter_id}")
                break
        
        if not interpreter_id:
            raise Exception(f"Interpreter '{interpreter_name}' が存在しますがリストに見つかりません")
    
    # READY ステータスを待機
    if status == "CREATING":
        print("⏳ Code Interpreter の準備完了を待機中...")
        for _ in range(30):
            check_response = agentcore_control.get_code_interpreter(codeInterpreterIdentifier=interpreter_id)
            status = check_response['status']
            if status == "READY":
                logger.info("✅ Code Interpreter が READY になりました")
                break
            elif status == "CREATE_FAILED":
                raise Exception("Code Interpreter の作成に失敗しました")
            time.sleep(10)
    
    return interpreter_id, interpreter_arn

# IAM ロールの伝播を待機
print("⏳ IAM ロールの伝播を待機中（10秒）...")
time.sleep(10)

# カスタム Code Interpreter を作成
print("\n🔧 カスタム Code Interpreter を作成中...")
CUSTOM_INTERPRETER_ID, CUSTOM_INTERPRETER_ARN = create_custom_code_interpreter(custom_role_arn)
print(f"✅ カスタム Code Interpreter を作成しました")
print(f"   ID: {CUSTOM_INTERPRETER_ID}")
print(f"   ARN: {CUSTOM_INTERPRETER_ARN}")
print(f"   ネットワークモード: PUBLIC")
print(f"   実行ロール: {custom_role_name}")

# SSM Parameter Store に保存
from lab_helpers.constants import PARAMETER_PATHS
ssm = boto3.client('ssm', region_name=AWS_REGION)
ssm.put_parameter(Name=PARAMETER_PATHS['lab_03']['code_interpreter_id'], Value=CUSTOM_INTERPRETER_ID, Type='String', Overwrite=True)
ssm.put_parameter(Name=PARAMETER_PATHS['lab_03']['code_interpreter_arn'], Value=CUSTOM_INTERPRETER_ARN, Type='String', Overwrite=True)
ssm.put_parameter(Name=PARAMETER_PATHS['lab_03']['code_interpreter_role_arn'], Value=custom_role_arn, Type='String', Overwrite=True)
print(f"✅ SSM Parameter Store に保存しました")

In [ ]:
ssm.get_parameter(Name=f'/{WORKSHOP_NAME}/lab-03/code-interpreter-id')['Parameter']['Value']

## 3.3 Code Interpreter クライアント関数の初期化と Code Interpreter セッションのテスト

In [ ]:
### 3.3: Code Interpreter クライアント関数を初期化

def initialize_code_interpreter_client():
    """AgentCore Code Interpreter クライアントを初期化"""
    global agentcore_code_interpreter, CODE_INTERPRETER_AVAILABLE
    
    try:
        agentcore_code_interpreter = boto3.client('bedrock-agentcore', region_name=AWS_REGION)
        CODE_INTERPRETER_AVAILABLE = True
        logger.info("✅ AgentCore Code Interpreter クライアントを初期化しました")
        return True
    except Exception as e:
        CODE_INTERPRETER_AVAILABLE = False
        logger.warning(f"⚠️ AgentCore Code Interpreter が利用できません: {e}")
        return False

def start_code_interpreter_session():
    """カスタム Interpreter を使用して Code Interpreter セッションを開始"""
    if not CODE_INTERPRETER_AVAILABLE:
        return None
    
    try:
        session_response = agentcore_code_interpreter.start_code_interpreter_session(
            codeInterpreterIdentifier=CUSTOM_INTERPRETER_ID,  # カスタム Interpreter を使用
            name=f"remediation-session-{uuid.uuid4()}",
            sessionTimeoutSeconds=1800  # 30分
        )
        
        session_id = session_response.get('sessionId')
        logger.info(f"✅ Code Interpreter セッションを開始しました: {session_id}")
        return session_id
        
    except Exception as e:
        logger.error(f"❌ Code Interpreter セッションの開始に失敗しました: {e}")
        return None

def stop_code_interpreter_session(session_id: str):
    """Code Interpreter セッションを停止"""
    if not session_id or not CODE_INTERPRETER_AVAILABLE:
        return
    
    try:
        agentcore_code_interpreter.stop_code_interpreter_session(
            codeInterpreterIdentifier=CUSTOM_INTERPRETER_ID,  # カスタム Interpreter を使用
            sessionId=session_id
        )
        logger.info(f"✅ Code Interpreter セッションを停止しました: {session_id}")
    except Exception as e:
        logger.error(f"❌ Code Interpreter セッションの停止に失敗しました: {e}")

def execute_remediation_code(session_id: str, code: str) -> Dict:
    """カスタム AgentCore Code Interpreter を使用して修復コードを実行"""
    if not session_id:
        return {"error": "Code Interpreter セッションが利用できません"}
    
    try:
        logger.info(f"🔧 修復コードを実行中: {code}")
        
        execute_response = agentcore_code_interpreter.invoke_code_interpreter(
            codeInterpreterIdentifier=CUSTOM_INTERPRETER_ID,  # カスタム Interpreter を使用
            sessionId=session_id,
            name="executeCode",
            arguments={
                "language": "python",
                "code": code
            }
        )
        
        # ストリーミングレスポンスを処理
        output_text = ""
        execution_status = "success"
        
        for event in execute_response.get('stream', []):
            if 'result' in event:
                result = event['result']
                if 'content' in result:
                    for content_item in result['content']:
                        if content_item.get('type') == 'text':
                            output_text += content_item.get('text', '')
                        elif content_item.get('type') == 'error':
                            execution_status = "error"
                            output_text += f"ERROR: {content_item.get('text', '')}"
        
        return {
            "execution_status": execution_status,
            "output": output_text,
            "session_id": session_id
        }
        
    except Exception as e:
        logger.error(f"❌ 修復コードの実行に失敗しました: {e}")
        return {"error": f"コード実行に失敗しました: {str(e)}"}

# 初期化をテスト
if initialize_code_interpreter_client():
    print("\n✅ カスタム Code Interpreter 統合の準備完了")
    print(f"   Interpreter ID: {CUSTOM_INTERPRETER_ID}")
    print(f"   ネットワークモード: PUBLIC")
    print(f"   実行ロール: {custom_role_name}")
    print(f"   クライアントステータス: {CODE_INTERPRETER_AVAILABLE}")
    print("   関数: initialize, start_session, stop_session, execute_code")
    
    # セッション作成をテスト
    print("\n🧪 セッション作成をテスト中...")
    test_session_id = start_code_interpreter_session()
    if test_session_id:
        print(f"✅ テストセッションを作成しました: {test_session_id}")
        stop_code_interpreter_session(test_session_id)
        print("✅ テストセッションを停止しました")
    else:
        print("❌ テストセッションの作成に失敗しました")
else:
    print("❌ Code Interpreter クライアントの初期化に失敗しました")
    print("   AWS 認証情報と AgentCore 権限を確認してください")

## 3.4 オブザーバビリティの有効化
AgentCore でオブザーバビリティを有効にするには、まずトランザクション検索を有効にする必要があります。これはリージョンごとに1回のセットアップです。
トランザクション検索は以下を提供します:
詳細な分析のための構造化ログとしてのスパン取り込み、セッション追跡のための X-Ray トレースインデックス作成、すべての AgentCore Runtime にわたる詳細なトレース分析
自分のアカウントでトランザクション検索を有効にする方法については、AWS ドキュメントを参照してください

In [ ]:
# 1. セッションをセットアップ
session = boto3.Session()
region = AWS_REGION
sts = session.client('sts')
account_id = sts.get_caller_identity()['Account']
logs_client = session.client('logs')
xray_client = session.client('xray')


print(f"アカウント: {account_id} のリージョン: {AWS_REGION} で AgentCore オブザーバビリティを設定中")

# ---------------------------------------------------------
# ステップ 1: リソースポリシー（べき等に近い）
# ---------------------------------------------------------
policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
        "Sid": "TransactionSearchXRayAccess",
        "Effect": "Allow",
        "Principal": { "Service": "xray.amazonaws.com" },
        "Action": "logs:PutLogEvents",
        "Resource": [
            f"arn:aws:logs:{region}:{account_id}:log-group:/aws/vendedlogs/xray:*",
            f"arn:aws:logs:{region}:{account_id}:log-group:aws/spans:*",
            f"arn:aws:logs:{region}:{account_id}:log-group:/aws/application-signals/*"
        ]
    }]
}

try:
    logs_client.put_resource_policy(
        policyName='BedrockAgentCoreXRayPolicy',
        policyDocument=json.dumps(policy_document)
    )
    print("✅ リソースポリシーが正常に作成/更新されました。")
except Exception as e:
    print(f"ℹ️ ポリシーに関する注意: {e}")

# ---------------------------------------------------------
# ステップ 2: トレース送信先を更新（「既に設定済み」を処理）
# ---------------------------------------------------------
try:
    xray_client.update_trace_segment_destination(Destination='CloudWatchLogs')
    print("✅ トレースセグメント送信先を CloudWatchLogs に設定しました。")
except xray_client.exceptions.InvalidRequestException as e:
    # エラーが単に既に有効になっているだけかどうかをチェック
    if "already set" in str(e):
        print("✅ トレース送信先は既に CloudWatchLogs に設定されていました（スキップ）。")
    else:
        print(f"❌ トレース送信先の更新エラー: {e}")

# ---------------------------------------------------------
# ステップ 3: インデックスルールを更新（修正: 文字列ではなく Dict を渡す）
# ---------------------------------------------------------
try:
    xray_client.update_indexing_rule(
        Name='Default',
        # 修正: json.dumps() 文字列ではなく Python 辞書を渡す
        Rule={"Probabilistic": {"DesiredSamplingPercentage": 5.0}}
    )
    print("✅ インデックスルールが正常に更新されました。")
except Exception as e:
    print(f"❌ インデックスルールの更新エラー: {e}")

In [ ]:
# ロググループ、ログソース、送信先を作成してアタッチ

def enable_observability_for_resource(resource_arn, resource_id, account_id, region=AWS_REGION):
    """
    Bedrock AgentCore リソース（例: Memory Store）のオブザーバビリティを有効化
    """
    logs_client = boto3.client('logs', region_name=region)

    # ステップ 0: ベンドログ配信用の新しいロググループを作成
    log_group_name = f'/aws/vendedlogs/bedrock-agentcore/{resource_id}'
    try:
        logs_client.create_log_group(logGroupName=log_group_name)
    except logs_client.exceptions.ResourceAlreadyExistsException:
        pass
    log_group_arn = f'arn:aws:logs:{region}:{account_id}:log-group:{log_group_name}'
    print(f"リソース CloudWatch ロググループ: f{log_group_arn}")

    unique_suffix = str(uuid.uuid4())[:8]
    
    # ステップ 1: ログ用の配信ソースを作成
    logs_source_response = logs_client.put_delivery_source(
        name=f"aiml301_custom_code_interpreter-{unique_suffix}-logs-source",
        logType="APPLICATION_LOGS",
        resourceArn=resource_arn
    )
    
    # ステップ 2: トレース用の配信ソースを作成  
    traces_source_response = logs_client.put_delivery_source(
        name=f"aiml301_custom_code_interpreter-{unique_suffix}-traces-source",
        logType="TRACES",
        resourceArn=resource_arn
    )
    
    # ステップ 3: 配信先を作成
    logs_destination_response = logs_client.put_delivery_destination(
        name=f"aiml301_custom_code_interpreter-{unique_suffix}-logs-dest",
        deliveryDestinationType='CWL',
        deliveryDestinationConfiguration={
            'destinationResourceArn': log_group_arn,
        }
    )
    
    # トレースは必須
    traces_destination_response = logs_client.put_delivery_destination(
        name=f"aiml301_custom_code_interpreter-{unique_suffix}-traces-dest",
        deliveryDestinationType='XRAY'
    )
    
    # ステップ 4: 配信を作成（ソースを送信先に接続）
    logs_delivery = logs_client.create_delivery(
        deliverySourceName=logs_source_response['deliverySource']['name'],
        deliveryDestinationArn=logs_destination_response['deliveryDestination']['arn']
    )
    
    # トレースは必須
    traces_delivery = logs_client.create_delivery(
        deliverySourceName=traces_source_response['deliverySource']['name'], 
        deliveryDestinationArn=traces_destination_response['deliveryDestination']['arn']
    )
    
    print(f"{resource_id} のオブザーバビリティを有効化しました")

# Code Interpreter リソース ID と ARN を取得
resource_id = ssm.get_parameter(Name=f'/{WORKSHOP_NAME}/lab-03/code-interpreter-id')['Parameter']['Value']
resource_arn = f"arn:aws:bedrock-agentcore:{AWS_REGION}:{account_id}:code-interpreter-custom/{resource_id}"

delivery_ids = enable_observability_for_resource(resource_arn, resource_id, account_id)

## 4. Strands エージェントツールの作成

**目標:** 承認ワークフロー付きの修復計画と実行のための Strands ツールを定義します。

**アプローチ:** プラン生成、実行、検証のための @tool デコレートされた関数を作成します。

**重要な学習ポイント:** 必須の承認ゲートを持つセキュアな修復ワークフローの実装方法。

In [ ]:
### 4.1: 修復ツールを定義 [execute_remediation_step]

@tool
def execute_remediation_step(remediation_code: str) -> str:
    """修復ステップを実行"""
    
    
    if not initialize_code_interpreter_client():
        return "AgentCore Code Interpreter が利用できません"
    
    session_id = start_code_interpreter_session()
    if not session_id:
        return "Code Interpreter セッションの開始に失敗しました"
    
    try:
        
        execution_result = execute_remediation_code(session_id, remediation_code)
        
        if 'error' in execution_result:
            return f"❌ 失敗: {execution_result['error']}"
        
        response = f"# ✅ 承認済み実行 - 結果\n\n"
        response += "## 実行出力\n\n```\n"
        response += execution_result['output']
        response += "\n```\n"
        
        return response
        
    except Exception as e:
        logger.error(f"❌ エラー: {e}")
        return f"❌ 修復プランの実行に失敗しました: {str(e)}"
    finally:
        stop_code_interpreter_session(session_id)

In [ ]:
### 4.2: 修復ツールを定義 [validate_remediation_environment]

@tool
def validate_remediation_environment() -> str:
    """修復環境が準備完了であることを検証"""
    
    logger.info("🔍 修復環境を検証中...")
    
    validation_results = {
        "code_interpreter_available": False,
        "session_creation": False,
        "aws_access": False,
        "environment_ready": False
    }
    
    try:
        # Code Interpreter の初期化をテスト
        if initialize_code_interpreter_client():
            validation_results["code_interpreter_available"] = True
            
            # セッション作成をテスト
            session_id = start_code_interpreter_session()
            if session_id:
                validation_results["session_creation"] = True
                validation_results["aws_access"] = True  # デモ用に簡略化
                stop_code_interpreter_session(session_id)
        
        validation_results["environment_ready"] = all([
            validation_results["code_interpreter_available"],
            validation_results["session_creation"],
            validation_results["aws_access"]
        ])
        
    except Exception as e:
        logger.error(f"❌ 環境検証に失敗しました: {e}")
    
    # レスポンスをフォーマット
    response = "# 修復環境検証\n\n"
    response += f"**検証日時**: {datetime.utcnow().isoformat()}\n\n"
    
    for check, status in validation_results.items():
        status_icon = "✅" if status else "❌"
        check_name = check.replace('_', ' ').title()
        response += f"- **{check_name}**: {status_icon} {'PASS' if status else 'FAIL'}\n"
    
    if validation_results["environment_ready"]:
        response += "\n🎉 **環境は修復の準備ができています**\n"
    else:
        response += "\n⚠️ **環境は準備ができていません**\n"
    
    return response

In [ ]:
### 4.3: 修復ツールを定義 [persist_remediation_scripts_to_s3]
@tool
def persist_remediation_scripts_to_s3(
    file_key: str,
    content: str
) -> dict:
    """Python スクリプトを S3 バケットに書き込む。
    
    Args:
        bucket_name: S3 バケット名
        file_key: ファイルが保存される S3 キー（パス/ファイル名）
        content: ファイルに書き込むコンテンツ
        region: AWS リージョン（デフォルト: us-west-2）
        content_type: コンテンツの MIME タイプ（デフォルト: text/plain）
    """
    bucket_name=retrieved_bucket_name
    region=AWS_REGION
    try:
        s3_client = boto3.client('s3', region_name=region)
        
        # S3 に書き込み
        s3_client.put_object(
            Bucket=bucket_name,
            Key=file_key,
            Body=content.encode('utf-8')
        )
        
        # S3 URL を生成
        s3_url = f"s3://{bucket_name}/{file_key}"
        https_url = f"https://{bucket_name}.s3.{region}.amazonaws.com/{file_key}"
        
        result = {
            "success": True,
            "message": "S3 へのファイル書き込みに成功しました",
            "bucket": bucket_name,
            "key": file_key,
            "s3_url": s3_url,
            "https_url": https_url,
            "size_bytes": len(content.encode('utf-8'))
        }
        
        return {
            "status": "success",
            "content": [
                {"text": f"✓ ファイルを {s3_url} に書き込みました"},
                {"json": result}
            ]
        }
        
    except Exception as e:
        error_msg = f"S3 へのファイル書き込みに失敗しました: {str(e)}"
        return {
            "status": "error",
            "content": [
                {"text": error_msg}
            ]
        }

In [ ]:
### 4.4: 修復ツールを定義 [read_remediation_scripts_from_s3]
@tool
def read_remediation_scripts_from_s3(prefix: str = "") -> dict:
    """S3 バケットからすべてのファイルを読み取り、その内容を返す。
    
    Args:
        prefix: ファイルをフィルタリングするオプションのプレフィックス（例: 'crm-remediation'）
    """
    bucket_name=retrieved_bucket_name
    region = AWS_REGION
    max_files = 100

    try:
        s3_client = boto3.client('s3', region_name=region)
        
        # オブジェクトをリスト
        list_params = {
            'Bucket': bucket_name,
            'MaxKeys': max_files
        }
        if prefix:
            list_params['Prefix'] = prefix
            
        response = s3_client.list_objects_v2(**list_params)
        

        if 'Contents' not in response:
            return {
                "status": "success",
                "content": [
                    {"text": f"s3://{bucket_name}/{prefix} にファイルが見つかりません"},
                    {"json": {
                        "success": True,
                        "bucket": bucket_name,
                        "prefix": prefix,
                        "file_count": 0,
                        "files": []
                    }}
                ]
            }
        
        files_data = []
        total_size = 0
        
        # 各ファイルを読み取り
        for obj in response['Contents']:
            file_key = obj['Key']
            
            # ディレクトリをスキップ（/ で終わるキー）
            if file_key.endswith('/'):
                continue
                
            try:
                # ファイルの内容を読み取り
                file_response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
                content = file_response['Body'].read().decode('utf-8')
                
                file_info = {
                    'key': file_key,
                    's3_url': f"s3://{bucket_name}/{file_key}",
                    'size': obj['Size'],
                    'last_modified': obj['LastModified'].isoformat(),
                    'content': content
                }
                files_data.append(file_info)
                total_size += obj['Size']
                
            except Exception as file_error:
                # ファイルが読み取れない場合、エラー情報を含めて続行
                files_data.append({
                    'key': file_key,
                    's3_url': f"s3://{bucket_name}/{file_key}",
                    'size': obj['Size'],
                    'last_modified': obj['LastModified'].isoformat(),
                    'error': str(file_error)
                })
        
        result = {
            "success": True,
            "message": f"S3 から {len(files_data)} 個のファイルを正常に読み取りました",
            "bucket": bucket_name,
            "prefix": prefix,
            "file_count": len(files_data),
            "total_size_bytes": total_size,
            "files": files_data
        }
        
        return {
            "status": "success",
            "content": [
                {"text": f"✓ s3://{bucket_name}/{prefix} から {len(files_data)} 個のファイルを読み取りました"},
                {"json": result}
            ]
        }
        
    except Exception as e:
        error_msg = f"S3 からのファイル読み取りに失敗しました: {str(e)}"
        return {
            "status": "error",
            "content": [
                {"text": error_msg}
            ]
        }

In [ ]:
# 現在のアプリケーションアーキテクチャと詳細を取得
with open('lab_helpers/lab_03/app_arch.txt', 'r') as f:
        current_app_architecture = f.read()

## 5. Strands エージェントの作成

**目標:** 修復ツールと適切なシステムプロンプトを持つ Strands エージェントを作成します。

**アプローチ:** Bedrock モデルと修復ツールでエージェントを設定します。

**重要な学習ポイント:** インフラストラクチャ修復ワークフロー用のエージェントを設定する方法。

In [ ]:
### 5.1: 修復ツールを持つ Strands エージェントを作成

def setup_agent(region=AWS_REGION):
    """修復ツールを持つ Strands エージェントをセットアップ"""
    try:
        if not initialize_code_interpreter_client():
            logger.error("❌ Code Interpreter クライアントの初期化に失敗しました")
            return None
        
        BOTO3_CONFIG = Config(
    read_timeout=300,      # モデルレスポンス用に5分
    connect_timeout=60,    # 接続用に1分
    retries={
        'max_attempts': 3,
        'mode': 'adaptive'
    }
)
        model = BedrockModel(
            model_id=MODEL_ID,
            streaming=True,
            max_tokens=4000,
            boto_client_config=BOTO3_CONFIG

        )
        system_prompt=f"""
            You are an AWS application remediation agent that helps in creating remediation plans in markdown format (no code execution). 
            Here are the application details and architecture: {current_app_architecture}
            Think step and step, break down the problems into smaller steps and use the persist_remediation_scripts_to_s3 tool to persist remediation plans.

FOCUS: Generate plans to restore system availability. No long-term improvements.

After the remediation plan has been created and persisted, provide the below summary:
1. **Issue Summary** - Brief description of the problem
2. **Root Cause** - Identified cause based on diagnostics
3. **Remediation Plan** - High level summary of the proposed fixes (numbered list)

        """
        agent = Agent(system_prompt=system_prompt,
            model=model, 
            tools=[execute_remediation_step, validate_remediation_environment, persist_remediation_scripts_to_s3, read_remediation_scripts_from_s3]
        )
        
        logger.info("✅ SRE 修復エージェントが Code Interpreter ツールで準備完了")
        logger.info(f"🌍 リージョン: {region}")
        logger.info(f"🔧 Code Interpreter 統合: {CODE_INTERPRETER_AVAILABLE}")
        
        return agent
        
    except Exception as e:
        logger.error(f"❌ エージェントのセットアップに失敗しました: {e}")
        return None

# エージェントをセットアップ
agent = setup_agent()
if agent:
    print("✅ Strands エージェントが正常に作成されました")
    print(f"   モデル: {MODEL_ID}")
    print(f"   ツール: 4 (execute_remediation_step, validate_remediation_environment, persist_remediation_scripts_to_s3, read_remediation_scripts_from_s3)")
    print(f"   Code Interpreter: {CODE_INTERPRETER_AVAILABLE}")
else:
    print("❌ エージェントのセットアップに失敗しました!")
    print("   Code Interpreter の初期化と AWS 認証情報を確認してください")

## 診断情報で Memory からコンテキストを補強

診断情報でコンテキストを補強するために、キュレーションされた Memory から追加情報を取得しましょう。

In [ ]:
agent_memory_client = boto3.client("bedrock-agentcore", region_name=AWS_REGION)

memory_id = get_parameter(PARAMETER_PATHS['memory']['memory_id'])
memory_session_id = get_parameter(PARAMETER_PATHS['memory']['default_session_id'])

print(memory_id)
print(memory_session_id)
actor_id='diagnostics_agent'

  

# Agent Memory に追加されたイベントをリストして、書き込みが成功したことを確認
params = {
                "memoryId": memory_id,
                "actorId": actor_id,
                "sessionId": memory_session_id,
                "includePayloads": True
            }
# すべてのメッセージを取得
response = agent_memory_client.list_events(**params)
additional_context=""
for event in response.get("events", []):
    payload = event.get('payload', [])
    for i, item in enumerate(payload):
        if 'conversational' in item:
            text = item['conversational']['content']['text']
            additional_context+=text
additional_context

## 6. 修復ワークフローのテスト

**目標:** 承認ゲートを持つ完全な修復ワークフローをデモンストレーションします。

**アプローチ:** 2段階の承認プロセスでインフラストラクチャ修復分析を実行します。

**重要な学習ポイント:** 計画から実行承認までのエンドツーエンドの修復プロセス。

In [ ]:
### 6.1: 完全な修復ワークフローを実行


if agent:
    print("🚀 完全な修復ワークフローを開始中...")
    print("=" * 60)
    print()
    
    # 修復プロンプトの例
    #remediation_prompt = f"""I need help with infrastructure remediation for our CRM application. We're experiencing: {additional_context} """

    remediation_prompt = f"""
    Help me fix  the dynamo DB throttling issues based on this diagnostic information: {additional_context}.
    
    """
    

    try:
         start_time = datetime.now()
         response = agent(remediation_prompt)
         analysis_time = (datetime.now() - start_time).total_seconds()
         
         print(f"\n🎯 修復分析結果:")
         print(f"分析時間: {analysis_time:.2f} 秒")
         
         # レスポンスを表示
         response_content = response.message.get('content', [])
         if response_content:
             for content in response_content:
                 if isinstance(content, dict) and 'text' in content:
                     text = content['text']
                     if len(text) > 2000:
                         print(f"\n📋 エージェント分析（最初の2000文字）:\n{text[:2000]}...")
                     else:
                         print(f"\n📋 エージェント分析:\n{text}")
    except Exception as e:
         print(f"❌ エラー: {e}")
    
    print("⚠️  注意: これは完全な修復計画ワークフローをデモンストレーションします")
    
else:
    print("❌ ワークフローのデモンストレーションにエージェントが利用できません!")

## 7. AgentCore Runtime へのデプロイ

**目標:** 修復エージェントをサーバーレス実行のために Amazon Bedrock AgentCore Runtime にデプロイします。

**アプローチ:** AgentCore 互換性のためにエージェントを変換し、CLI を使用してデプロイします。

**重要な学習ポイント:** Code Interpreter 統合を持つ Strands エージェントを本番環境対応のサーバーレスインフラストラクチャにデプロイする方法。

### 7.1: AgentCore 互換エージェントの作成

まず、必要なラッパーとエントリポイントを持つ修復エージェントの AgentCore 互換バージョンを作成する必要があります。

In [ ]:
### 7.1: カスタム Runtime IAM ロールを作成

# Deployer を初期化
deployer = AgentCoreRuntimeDeployer(region=AWS_REGION, prefix=WORKSHOP_NAME, verbose=False)

# AgentCore スターターキットの前提条件を確認
if not deployer.check_prerequisites():
    raise RuntimeError("前提条件が満たされていません。インストール: pip install bedrock-agentcore-starter-toolkit")

# カスタムポリシーを読み込み、プレースホルダーを置換
iam = boto3.client('iam')
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
role_name = f"{WORKSHOP_NAME}_CustomRemediationRuntimeRole"

# 信頼ポリシーを読み込み
with open('lab_helpers/lab_03/custom_runtime_trust_policy.json', 'r') as f:
    trust_policy = f.read()
    trust_policy = trust_policy.replace('{{ACCOUNT_ID}}', account_id)
    trust_policy = trust_policy.replace('{{REGION}}', AWS_REGION)

# 権限ポリシーを読み込み
with open('lab_helpers/lab_03/custom_runtime_permissions.json', 'r') as f:
    permissions_policy = f.read()
    permissions_policy = permissions_policy.replace('{{ACCOUNT_ID}}', account_id)
    permissions_policy = permissions_policy.replace('{{REGION}}', AWS_REGION)
    permissions_policy = permissions_policy.replace('{{PREFIX}}', WORKSHOP_NAME)

# ロールを作成または更新
try:
    role = iam.get_role(RoleName=role_name)
    role_arn = role['Role']['Arn']
    iam.update_assume_role_policy(RoleName=role_name, PolicyDocument=trust_policy)
    print(f"✅ 既存のロールを使用: {role_name}")
except iam.exceptions.NoSuchEntityException:
    role = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=trust_policy,
        Description="Custom execution role for AgentCore Runtime"
    )
    role_arn = role['Role']['Arn']
    print(f"✅ カスタムロールを作成しました: {role_name}")
    import time
    time.sleep(10)

# 権限をアタッチ
iam.put_role_policy(
    RoleName=role_name,
    PolicyName=f"{WORKSHOP_NAME}_RuntimePermissions",
    PolicyDocument=permissions_policy
)

role_info = {'role_arn': role_arn, 'role_name': role_name}
print(f"✅ カスタム権限をアタッチしました")
print(f"   ロール ARN: {role_arn}")

In [ ]:
### 7.1: デプロイ用のエージェントコード

# ヘルパーファイルからエージェントコードを読み込み
with open('lab_helpers/lab_03/runtime_mcp_agent_code.py', 'r') as f:
    agentcore_agent_code = f.read()

print("✅ エージェントコードを読み込みました: lab_helpers/lab_03/runtime_mcp_agent_code.py")
print(f"   コード長: {len(agentcore_agent_code)} 文字")
print(f"   トランスポート: streamable-http（AgentCore Runtime 互換）")

In [ ]:
### 7.1: エージェントコードをディスクに書き込み
with open('agent-remediation.py', 'w') as f:
      f.write(agentcore_agent_code)

print(f"✅ エージェントコードを書き込みました: agent-remediation.py")

### 7.2 JWT Authorizer で Runtime を設定

**runtime.configure() の機能:**
- エージェントコードと依存関係を検証
- Dockerfile と AWS 設定ファイルを生成
- デプロイメントブループリントを準備（ローカル操作、AWS リソースはまだ作成されません）
- 実行ロールとトークン検証をセットアップ

**JWT Authorizer の設定:**
`authorizer_configuration` パラメータは、Runtime が受信トークンを検証する方法を指示します:
- **discoveryUrl**: Runtime が署名検証用の公開鍵を取得する Cognito OIDC エンドポイント
- **allowedClients**: User Auth クライアント（直接ユーザー）と M2M クライアント（Gateway）の両方が許可されます

**自動的な Runtime の動作:**
```
リクエストの Bearer トークン → 署名を検証 → 発行者を確認（Cognito）→ allowedClients でクライアント ID を検証 → 許可または拒否
```

In [ ]:
from lab_helpers.parameter_store import get_parameter
from lab_helpers.constants import PARAMETER_PATHS

# Lab-01 の SSM Parameter Store から Cognito 設定を取得
user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])
m2m_client_id = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_id'])
user_auth_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])

# Cognito ディスカバリー URL を構築 - Runtime がトークン検証用の公開鍵を取得するために使用
discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"

print(f"✅ Cognito 設定を取得しました")
print(f"   ディスカバリー URL: {discovery_url}")
print(f"   許可されたクライアント: User Auth + M2M")
print(f"   User Auth クライアント ID: {user_auth_client_id}")
print(f"   M2M クライアント ID: {m2m_client_id}")

In [ ]:
### 7.2 b: JWT Authorizer で Runtime を設定

# Runtime オブジェクトを初期化
runtime = Runtime()

# JWT Authorizer 設定
# - discoveryUrl: Cognito OIDC エンドポイント（Runtime が自動的に公開鍵を取得）
# - allowedClients: User Auth と M2M の両方のクライアントが Runtime を呼び出し可能
authorizer_config = {
    "customJWTAuthorizer": {
        "discoveryUrl": discovery_url,
        "allowedClients": [user_auth_client_id, m2m_client_id]
    }
}

# 重要: JWT トークン検証で Runtime を設定
print(f"\n🔍 使用する実行ロール: {role_info['role_arn']}")
print(f"   ロール名: {role_info['role_name']}")

runtime.configure(
      entrypoint='agent-remediation.py',
      execution_role=role_info['role_arn'],
      auto_create_ecr=True,
      requirements_file='requirements.txt',
      region=AWS_REGION,
      agent_name=f"{WORKSHOP_NAME}_remediation_runtime",
      protocol="MCP",
      authorizer_configuration=authorizer_config  # ← トークン検証を有効化
)

print("✅ JWT Authorizer で Runtime を設定しました")
print(f"   プロトコル: MCP")
print(f"   JWT トークン検証: 有効")
print(f"   許可されたトークン: User Auth + M2M")

### 7.3: AgentCore に Runtime を起動

Python SDK の `runtime.launch()` を使用して、設定した Runtime を AgentCore にデプロイします。この重要なステップで、ローカルエージェントを本番サーバーレスサービスに変換します。

#### 7.3 a: 起動プロセスの理解

  **runtime.launch() 実行中に起こること:**

  1️⃣ CodeBuild が Docker コンテナのビルドを開始

  2️⃣ requirements.txt の依存関係がインストールされる

  3️⃣ イメージが Amazon ECR にプッシュされる（自動作成）

  4️⃣ AgentCore が Runtime を MCP サービスとして登録
  
  5️⃣ CloudWatch ログが設定される

  ⏱️ **典型的な所要時間:** 5-10分

In [ ]:
# すべてのセルを実行するシナリオに対応するために追加
import time
time.sleep(10)

In [ ]:
### 7.3 b: runtime.launch() を実行

print("\n🚀 Runtime を AgentCore に起動中...\n")

try:
    # 競合時に自動更新で Runtime を起動
    # これは同期的 - CodeBuild と初期ヘルスチェックを待機
    launch_result = runtime.launch(auto_update_on_conflict=True)
    
    # デプロイ ARN を抽出
    runtime_arn = launch_result.agent_arn
    
    print(f"✅ Runtime が正常に起動しました!")
    print(f"   Runtime ARN: {runtime_arn}")
    print(f"\n📝 次のセクション用に設定が保存されました:")
    
except Exception as e:   
    print(f"❌ 起動に失敗しました: {e}")
    print(f"\nトラブルシューティング:")
    print(f"  • CodeBuild サービス制限を確認")
    print(f"  • IAM ロールの ECR 権限を確認")
    print(f"  • CloudWatch ログでビルドエラーを確認")
    print(f"  • requirements.txt のすべての依存関係が正しいことを確認")
    raise

In [ ]:
### 7.3c: Runtime 設定を保存

# Runtime 設定を抽出して保存
runtime_arn = launch_result.agent_arn
runtime_id = getattr(launch_result, 'agent_id', None)

# 設定を保存
from lab_helpers.lab_03 import store_runtime_configuration
store_runtime_configuration(runtime_arn, runtime_id, region=AWS_REGION, prefix=WORKSHOP_NAME)

print("\n✅ Runtime がデプロイされ、設定されました")
print(f"   ARN: {runtime_arn}")
print(f"   Gateway 登録の準備完了")

In [ ]:
# ロギング設定ヘルパーをインポート
from lab_helpers.lab_03.configure_logging import configure_runtime_logging

# Runtime 用の CloudWatch Logs 配信を設定
logging_config = configure_runtime_logging(
      runtime_arn=runtime_arn,  # 前のセルから
      runtime_id=runtime_id,     # 前のセルから
      region=AWS_REGION,
      log_type="APPLICATION_LOGS"  # コンテナの stdout/stderr ログ
  )

print("\n📊 ロギング設定サマリー:")
print(f"  ロググループ: {logging_config['log_group_name']}")
print(f"  配信ステータス: {logging_config['delivery_status']}")
print(f"  配信 ID: {logging_config['delivery_id']}")

## 8. Cognito JWT 認可で Gateway をデプロイ

**目標:** 受信ユーザー認証用の Cognito JWT Authorizer を持つ AgentCore Gateway をデプロイします。

**アプローチ:** Cognito User Auth Client と JWT 検証を Gateway アクセス制御に使用します。

**重要な学習ポイント:** Cognito 認証を AgentCore Gateway に統合する方法。

In [ ]:
# Lab-01 から Cognito 認証情報を取得
user_auth_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])
user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])

# Cognito OIDC ディスカバリー URL を構築
discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"

# ヘルパーを使用して Gateway IAM ロールを作成
gateway_setup = AgentCoreGatewaySetup(region=AWS_REGION, prefix=WORKSHOP_NAME, verbose=False)
role_info = gateway_setup.create_gateway_service_role()
role_arn = role_info['role_arn']

# boto3 で直接 Gateway を作成（シンプルな API 呼び出し）
agentcore = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

gateway_response = agentcore.create_gateway(
      name=f"aiml301-remediation-gateway",
      roleArn=role_arn,
      protocolType="MCP",
      authorizerType="CUSTOM_JWT",
      authorizerConfiguration={
          "customJWTAuthorizer": {
              "discoveryUrl": discovery_url,
              "allowedClients": [user_auth_client_id]
          }
      }
  )

gateway_id = gateway_response['gatewayId']
gateway_url = gateway_response['gatewayUrl']

# 設定を保存
put_parameter(PARAMETER_PATHS['lab_03']['gateway_id'], gateway_id)
put_parameter(PARAMETER_PATHS['lab_03']['gateway_role_arn'], role_arn)

print("✅ JWT 認可で Gateway をデプロイしました")
print(f"   Gateway ID: {gateway_id}")
print(f"   Gateway URL: {gateway_url}")
print(f"   認証タイプ: Cognito JWT")

## 9. M2M 認証で Runtime を Gateway ターゲットとして追加

  **目標:** OAuth2 M2M 認証で Runtime を Gateway ターゲットとして登録します。

  **アーキテクチャ:**

```
  User（JWT）→ Gateway（JWT 検証）→ Runtime（M2M トークン）
                      ↓
              Gateway が自動的に:
              - GetResourceOauth2Token を呼び出し
              - Secrets Manager から認証情報を取得
              - Cognito から M2M アクセストークンを取得
              - リクエストに Bearer トークンを注入
                      ↓
              Authorization: Bearer {token} でRuntime を呼び出し
```
  **Gateway が自動的に M2M OAuth2 トークンを送信する方法:**

  1. **認証情報プロバイダーストレージ**
     - OAuth2 認証情報プロバイダーが `clientId` + `clientSecret` を AWS Secrets Manager に保存（暗号化）
     - プロバイダー ARN がこのセキュアなストレージ場所を指す

  2. **自動トークン取得フロー**
  ```
     credentialProviderConfigurations で Gateway ターゲットを作成
         ↓
     Gateway が Runtime ターゲットを呼び出す必要がある
         ↓
     Gateway が GetResourceOauth2Token API を呼び出し（自動、組み込み）
         ↓
     AgentCore Identity が Secrets Manager から client_id + client_secret を取得
         ↓
     AgentCore が Cognito トークンエンドポイントを呼び出し:
     POST /token
     Body: grant_type=client_credentials&client_id=...&client_secret=...&scope=...
         ↓
     Cognito が M2M アクセストークンを返す
         ↓
     AgentCore がトークンをキャッシュし、更新ライフサイクルを管理
         ↓
     Gateway がリクエストヘッダーにトークンを注入:
     Authorization: Bearer {access_token}
         ↓
     Gateway が Bearer トークン付きで Runtime MCP エンドポイントを呼び出し
         ↓
     Runtime が Cognito 公開鍵（JWKS）を使用して JWT 署名を検証
```
  3. **カスタムコード不要**
  - すべての認証情報管理が自動
  - トークン更新が自動
  - トークン注入が自動
  - コードでは認証情報プロバイダー ARN を指定するだけ

  **重要な学習ポイント:** デュアル認証 - ユーザーベースの受信（JWT）、サービスベースの送信（OAuth2 M2M）。

In [ ]:
# すべてのセルを実行するシナリオに対応するために追加
import time
time.sleep(10)

In [ ]:
### 9.1 セキュアアクセス用の AgentCore Identity CredentialsProvider を作成
# Lab-01 Cognito セットアップから M2M 認証情報を取得
m2m_client_id = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_id'])
m2m_client_secret = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_secret'])
user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])

# Cognito OIDC ディスカバリー URL を構築
discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"

# AgentCore クライアントを初期化
agentcore = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

# OAuth2 認証情報プロバイダーを作成
# M2M 認証情報を Secrets Manager に安全に保存
credential_provider_response = agentcore.create_oauth2_credential_provider(
      name='aiml301-m2m-credentials',
      credentialProviderVendor='CustomOauth2',
      oauth2ProviderConfigInput={
          'customOauth2ProviderConfig': {
              'clientId': m2m_client_id,
              'clientSecret': m2m_client_secret,
              'oauthDiscovery': {
                  'discoveryUrl': discovery_url
              }
          }
      }
  )

# レスポンスを抽出
oauth2_provider_arn = credential_provider_response['credentialProviderArn']
client_secret_arn = credential_provider_response['clientSecretArn']['secretArn']

# 次のセクション用に SSM に保存
put_parameter(PARAMETER_PATHS['lab_03']['oauth2_provider_arn'], oauth2_provider_arn)
put_parameter(PARAMETER_PATHS['lab_03']['oauth2_secret_arn'], client_secret_arn)

print("✅ OAuth2 認証情報プロバイダーを作成しました")
print(f"\n📋 認証情報ストレージ:")
print(f"   プロバイダー ARN: {oauth2_provider_arn}")
print(f"   シークレット ARN: {client_secret_arn}")
print(f"   場所: AWS Secrets Manager（暗号化）")
print(f"   認証情報: M2M client_id + client_secret")

In [ ]:
### 9.2 M2M OAuth2 で Runtime 用のターゲットを作成

import urllib.parse

# SSM から設定を取得
gateway_id = get_parameter(PARAMETER_PATHS['lab_03']['gateway_id'], region_name=AWS_REGION)
runtime_arn = get_parameter(PARAMETER_PATHS['lab_03']['runtime_arn'], region_name=AWS_REGION)
oauth2_provider_arn = get_parameter(PARAMETER_PATHS['lab_03']['oauth2_provider_arn'], region_name=AWS_REGION)
resource_server_id = get_parameter(PARAMETER_PATHS['cognito']['resource_server_identifier'], region_name=AWS_REGION)

# 重要: Runtime ARN からエンドポイント URL を構築
# 形式: https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{URL_ENCODED_ARN}/invocations?qualifier=DEFAULT
# ARN 例: arn:aws:bedrock-agentcore:us-west-2:123456789012:runtime/my-runtime
encoded_arn = urllib.parse.quote(runtime_arn, safe='')
endpoint_url = f"https://bedrock-agentcore.{AWS_REGION}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

# きめ細かい認可用の M2M スコープ
m2m_scopes = [
    f"{resource_server_id}/mcp.invoke",
    f"{resource_server_id}/runtime.access"
]

# AgentCore クライアントを初期化
agentcore = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

# Gateway ターゲットを作成 - M2M OAuth2 を持つ外部 MCP サーバーとしての Runtime
# Gateway は自動的に:
# 1. oauth2_provider_arn を使用して Cognito から M2M トークンを取得
# 2. Runtime を呼び出すときに Bearer トークンを含める
# 3. Runtime がトークンを検証し、スコープに基づいて操作を認可
target_response = agentcore.create_gateway_target(
    gatewayIdentifier=gateway_id,
    name="aiml301-runtime-target",
    description="AgentCore Runtime with M2M OAuth2 authentication",
    targetConfiguration={
        "mcp": {
            "mcpServer": {
                "endpoint": endpoint_url  # ← URL エンコードで Runtime ARN から構築
            }
        }
    },
    credentialProviderConfigurations=[
        {
            "credentialProviderType": "OAUTH",
            "credentialProvider": {
                "oauthCredentialProvider": {
                    "providerArn": oauth2_provider_arn,  # ← セクション 9.1 の OAuth2 認証情報プロバイダーを参照
                    "scopes": m2m_scopes
                }
            }
        }
    ]
)

target_id = target_response['targetId']
put_parameter(PARAMETER_PATHS['lab_03']['gateway_runtime_target'], target_id, region_name=AWS_REGION)

print("✅ M2M OAuth2 で Runtime を Gateway ターゲットとして追加しました")
print(f"   ターゲット ID: {target_id}")
print(f"   Runtime ARN: {runtime_arn}")
print(f"   エンドポイント: {endpoint_url}")
print(f"   認証情報プロバイダー: {oauth2_provider_arn}")

In [ ]:
import time

### 9.2 ターゲットステータスを確認して同期

# ターゲットが READY になるのを待機
print("\n⏳ ターゲットが READY になるのを待機中...")
for attempt in range(30):
      target_info = agentcore.get_gateway_target(
          gatewayIdentifier=gateway_id,
          targetId=target_id
      )
      status = target_info.get('status', 'UNKNOWN')

      if status == 'READY':
          print(f"✅ ターゲットが READY になりました")
          break
      if status == 'FAILED' or status == 'SYNCHRONIZE_UNSUCCESSFUL':
          print(f"❌ ターゲットがエラー状態: {target_info.get('statusReasons', 'エラーメッセージなし')}")
          break
      time.sleep(5)

# ツールを検出するために同期
agentcore.synchronize_gateway_targets(
      gatewayIdentifier=gateway_id,
      targetIdList=[target_id]
  )

print("\n✅ 完了 - Gateway が自動的に M2M トークンを管理します")

## 10. JWT 認証での MCP クライアント統合

**目標:** Cognito 認証経由でローカル MCP クライアントを Gateway に接続します。

**アプローチ:**
1. Cognito からトークンを取得
2. ローカル MCP サーバーラッパーを作成
3. MCP プロトコルで Gateway に接続
4. エンドツーエンドフローをテスト

**重要な学習ポイント:** 完全な統合: ローカルクライアント → MCP → Gateway → Runtime → Strands エージェント → 修復

In [ ]:
### 10.1: テストユーザー認証情報を表示
# SSM Parameter Store からテストユーザー認証情報を取得
user_pool_id = get_parameter(PARAMETER_PATHS["cognito"]["user_pool_id"])
token_endpoint = get_parameter(PARAMETER_PATHS["cognito"]["token_endpoint"])
user_client_id = get_parameter(PARAMETER_PATHS["cognito"]["user_auth_client_id"])
test_username = get_parameter(PARAMETER_PATHS["cognito"]["test_user_email"])
test_password = get_parameter(PARAMETER_PATHS["cognito"]["test_user_password"])

print(f"  ✓ User Pool: {user_pool_id}")
print(f"  ✓ クライアント ID: {user_client_id}")
print(f"  ✓ ユーザー名: {test_username}")

In [ ]:
### 10.2: テストユーザーを認証して JWT トークンを取得
print("\n🔑 Cognito で認証中...")

cognito = boto3.client('cognito-idp', region_name=AWS_REGION)

response = cognito.initiate_auth(
      ClientId=user_client_id,
      AuthFlow='USER_PASSWORD_AUTH',
      AuthParameters={
          'USERNAME': test_username,
          'PASSWORD': test_password
      }
  )

access_token = response['AuthenticationResult']['AccessToken']
id_token = response['AuthenticationResult']['IdToken']
expires_in = response['AuthenticationResult']['ExpiresIn']

print(f"  ✅ 認証に成功しました!")
print(f"  ✓ トークンタイプ: Bearer")
print(f"  ✓ 有効期限: {expires_in} 秒")
print(f"  ✓ アクセストークン（最初の50文字）: {access_token[:50]}...")

In [ ]:
### 10.3: JWT トークンをデコードしてクレームを表示

import base64
# JWT トークンをデコードしてクレームを表示
print("\n📋 JWT トークンをデコード中...")

# JWT トークンは3つの部分で構成: header.payload.signature
parts = access_token.split('.')
if len(parts) == 3:
      # ペイロードをデコード（必要に応じてパディングを追加）
      payload_b64 = parts[1]
      # base64 デコード用にパディングを追加
      padding = 4 - len(payload_b64) % 4
      if padding != 4:
          payload_b64 += '=' * padding

payload_json = base64.urlsafe_b64decode(payload_b64)
payload = json.loads(payload_json)

print("  トークンクレーム:")
print(f"    • Subject (sub): {payload.get('sub', 'N/A')}")
print(f"    • ユーザー名: {payload.get('username', 'N/A')}")
print(f"    • クライアント ID: {payload.get('client_id', 'N/A')}")
print(f"    • トークン用途: {payload.get('token_use', 'N/A')}")
print(f"    • スコープ: {payload.get('scope', 'N/A')}")
print(f"    • 発行日時: {payload.get('iat', 'N/A')}")
print(f"    • 有効期限: {payload.get('exp', 'N/A')}")

In [ ]:
### 10.4: MCP クライアントを使用して AgentCore Gateway に接続

from lab_helpers.lab_03.mcp_client import MCPClient

print("=" * 80)
print("🌉 AgentCore Gateway に接続中")
print("=" * 80)

print(f"  ✓ Gateway URL: {gateway_url}")

# MCP クライアントを作成
client = MCPClient(gateway_url, access_token)
# セッションを初期化
client.initialize()

In [ ]:
### 10.5: Gateway から利用可能なツールを一覧表示
print("=" * 80)
print("🔧 ステップ 4: 利用可能な MCP ツールを一覧表示")
print("=" * 80)

# Gateway 経由で利用可能なすべてのツールを一覧表示
tools = client.list_tools()

# 簡単にアクセスできるようにツール名を保存
tool_names = [tool['name'] for tool in tools]
print(f"\n📝 利用可能なツール: {tool_names}")

In [ ]:
print("=" * 80)
print("🔍 ステップ 5: ツール呼び出しをテスト")
print("=" * 80)

# ddgs_search ツールを検索
search_tool = next((t for t in tools if 'infrastructure_agent' in t['name'].lower() and 'news' not in t['name'].lower()), None)

start_time = time.time()
try:
    if search_tool:
        print(f"\n🎯 使用するツール: {search_tool['name']}")

        # 検索ツールを呼び出し
        result = client.call_tool(
        tool_name=search_tool['name'],
        arguments={
            "remediation_query": f"""I need help with infrastructure remediation for our CRM application. We're experiencing: {additional_context} """,
            "action_type": "only_plan"
        }
    )

            

        print("\n✅ エンドツーエンドテスト完了!")
        print("   クライアント → Gateway → Runtime → MCP サーバー → 修復エージェント ✓")
    else:
        print("❌ 検索ツールが見つかりません")

except Exception as e:
    print(f"❌ エラー: {e}")

end_time = time.time()
print(f"  ⏰ 合計所要時間: {end_time - start_time:.2f} 秒")

## 11. クリーンアップ

**目標:** カスタム Code Interpreter と IAM ロールを含むすべての Lab 03 リソースを削除します。

**重要:** Lab-04 を次に実行する予定がない場合のみクリーンアップを実行してください。

In [ ]:
### 11.1: Lab 03 のすべてのリソースをクリーンアップ

from lab_helpers.lab_03 import cleanup_lab_03
from lab_helpers.config import AWS_REGION

# 次に Lab-04 を実行する予定がない場合のみ実行

#cleanup_lab_03(region_name=AWS_REGION, verbose=True)

**次のステップ:**
- ✅ すべての Lab 03 リソースが削除されました
- ✅ AgentCore Runtime と Gateway の AWS コストがゼロになりました
- ✅ 他の Lab を安全に実行するか、Lab 03 を再実行できます

**Lab 03 を再実行する場合:**
1. セクション 1（インポート）から開始
2. すべてのヘルパー関数は引き続き利用可能
3. IAM ロールは新しい権限で再作成されます

**Lab 04 に進む場合:**
- 予防エージェントワークフローについては `Lab-04-prevention-agent.ipynb` を参照

## まとめ: Lab 3 - 修復エージェントアーキテクチャ

✅ **完了:**
1. ✓ **カスタム Code Interpreter セットアップ** - IAM 実行ロール付きのカスタム Interpreter を作成
2. ✓ **カスタム IAM ロール** - CloudWatch、S3、X-Ray、Metrics の権限を設定
3. ✓ **PUBLIC ネットワークモード** - VPC 要件なしの簡素化されたセットアップ
4. ✓ **セッション管理** - 適切なクリーンアップ付きの Code Interpreter セッションの開始/停止
5. ✓ **修復ツール** - 承認ワークフロー付きの計画、実行、検証ツール
6. ✓ **Strands エージェント** - 包括的なシステムプロンプト付きのエキスパート修復エージェント
7. ✓ **AgentCore Runtime デプロイ** - 本番環境対応のサーバーレスデプロイ
8. ✓ **本番テスト** - デプロイされたエージェントの機能とセキュリティを検証
9. ✓ **完全なクリーンアップ** - カスタム Interpreter と IAM ロールのクリーンアップを含む

**完全なワークフロー:**
```
カスタム Code Interpreter セットアップ
    ├─ IAM 実行ロール ✓
    ├─ カスタム Interpreter（PUBLIC モード）✓
    └─ セッションテスト ✓
    ↓
開発（ノートブック）
    ↓
ローカルテストと検証
    ↓
AgentCore Runtime デプロイ
    ├─ CodeBuild コンテナビルド ✓
    ├─ AWS リソース作成 ✓
    └─ サーバーレスデプロイ ✓
    ↓
本番エージェント
    ├─ サーバーレス実行 ✓
    ├─ 自動スケーリング ✓
    ├─ カスタム Code Interpreter 統合 ✓
    ├─ 承認ワークフロー ✓
    └─ モニタリングとロギング ✓
    ↓
クリーンアップ
    ├─ カスタム Code Interpreter ✓
    ├─ IAM 実行ロール ✓
    └─ すべての Lab リソース ✓
```
**本番ステータス: ✅ デプロイ済みで稼働中**

**主要機能:**
- カスタマイズされた権限を持つカスタム Code Interpreter
- 簡素化されたセットアップのための PUBLIC ネットワークモード
- 実行環境の完全な制御
- 完全なリソースクリーンアップを含む

**次へ: Lab 4 - 予防エージェント** (`Lab-04-prevention-agent.ipynb`)
- AgentCore Browser を使用したプロアクティブなインフラストラクチャ分析
- リアルタイムの AWS ベストプラクティス調査
- 問題が発生する前に回避するための予防重視の推奨事項
- 完全な SRE 自動化パイプライン: 予防 + 修復